In [1]:
from syft.core.node.new.action_object import ActionObject, ActionObjectPointer
from syft.core.node.new.action_service import NumpyArrayObject
from syft.core.node.common.node_table.syft_object import SyftObject, transform
from syft.core.common.uid import UID
from syft.core.common.serde.serializable import serializable
from typing import Optional, Tuple, Dict, List, Any
import numpy as np
from result import Err
from result import Ok
from result import Result

import pydantic
from pydantic import PrivateAttr

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft
/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-16 13:13:58.147233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-01-16 13:13:58.161261: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-16 13:13:58.591594: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou

In [15]:
@serializable(recursive_serde=True)
class DatasetPointer(ActionObjectPointer):
    __canonical_name__ = "DatasetPointer"
    __version__ = 1
    
    __attr_state__ = ["id", "node_uid", "items"]
    
    def __post_init__(self) -> None:
        self.setup_methods()
        
    def setup_methods(self) -> None:
        return

@serializable(recursive_serde=True)
class DatasetItem(ActionObject):
    __canonical_name__ = "DatasetItem"
    __version__ = 1
    
    id: UID
    name: str
    dtype: str
    shape: Optional[Tuple]
    # data: Any
    _syft_action_data: Any = (
        PrivateAttr()
    )
    
    __attr_state__ = [
        "id",
        "name",
        "dtype",
        "shape",
    ]
    __attr_searchable__ = ["name"]

@serializable(recursive_serde=True)
class Dataset:#(ActionObject):#(SyftObject):
    __canonical_name__ = "DatasetObject"
    __version__ = 1
    
    id: UID
    name: str
    description: str
    tags: List[str] = []
    metadata: Dict = {}# TODO: decide how to restructure this
    items: Dict[str, DatasetItem]
    # _syft_action_data: Any = (
    #     PrivateAttr()
    # )
    
    __attr_searchable__ = ["name", "description", "tags"]
    __attr_unique__ = ["name"]

    pointer_type = DatasetPointer
    
    def __init__(
        self, 
        id: UID,
        name: str, 
        description: str, 
        items: Dict[str, DatasetItem], 
        tags: Optional[List[str]] = [], 
        metadata: Optional[Dict] = {}
    ):
        self.id = id
        self.name = name
        self.description = description
        self.items = items
        self.tags = tags
        self.metadata = metadata
        
    def __getitem__(self, key:str) -> Any:
        if key in self.items:
            return Ok(self.items[key])
        else:
            return Err(f'Key {key} not in store')



In [16]:
raw_data = np.array([1, 2, 3])
item = DatasetItem(
        name='item 1', 
        id=UID(), 
        dtype=str(np.ndarray), 
        _syft_action_data=raw_data
    )
dataset = Dataset(
            name="custom dataset", 
            id=UID(), 
            items={
                "item 1": item
            }, 
            description="This is our custom dataset"
        )

In [19]:
dataset["item 2"]

Err('Key item 2 not in store')